In [ ]:
%%capture
!pip install --upgrade pip
!pip install huggingface_hub
!pip install datasets[audio]
!pip install evaluate
!pip install git+https://github.com/huggingface/transformers
!pip install torchaudio
!pip install librosa
!pip install jiwer
!pip install accelerate=0.34.0

In [ ]:
!nvidia-smi

Wed Sep 25 19:38:56 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from google.colab import drive

drive.mount('/content/drive/', force_remount=True)

In [ ]:
import pandas as pd
import os
from datasets import load_dataset, Audio, Value, Features

data_dir = 'task1'

docs_dir = os.path.join(data_dir, 'docs')
wav_dir = os.path.join(data_dir, 'wav')
train_labels_path = os.path.join(docs_dir, 'train_labels.txt')

csv_save_dir = 'deepmine_csv'

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv(train_labels_path, sep=' ', header=0)
label_list = df['speaker-id'].unique().tolist()
label_list.sort()  # Let's sort it for determinism
num_labels = len(label_list)
print(f"A classification problem with {num_labels}")

label2id={label: i for i, label in enumerate(label_list)}
id2label={i: label for i, label in enumerate(label_list)}

df['label'] = df['speaker-id'].map(label2id)
df['path'] = f'task1/wav/train/' + df['train-file-id'] + '.wav'

df = df.sample(frac=1).reset_index(drop=True)
df = df.drop(columns=['phrase-id'])
df.iloc[0].path

A classification problem with 1620


'task1/wav/train/trn_140598.wav'

In [ ]:
from datasets import load_dataset

!mkdir -p /content/data
save_path = "/content/data"

df = df.reset_index(drop=True)
df.to_csv(f"{save_path}/train.csv", sep="\t", encoding="utf-8", index=False)
print(df.shape)

data_files = {
    "train": "/content/data/train.csv",
}

dataset = load_dataset("csv", data_files=data_files, delimiter="\t", )
train_dataset = dataset["train"]
print(train_dataset)

(183431, 4)


Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['train-file-id', 'speaker-id', 'label', 'path'],
    num_rows: 183431
})


In [ ]:

!mkdir -p /content/data

save_path = "/content/data"

train_df, test_df = train_test_split(df, test_size=0.05, random_state=42, stratify=df["label"])

train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

train_df.to_csv(f"{save_path}/train.csv", sep="\t", encoding="utf-8", index=False)
test_df.to_csv(f"{save_path}/test.csv", sep="\t", encoding="utf-8", index=False)

print(train_df.shape)
print(test_df.shape)

(174259, 3)
(9172, 3)


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['train-file-id', 'speaker-id', 'label'],
    num_rows: 174259
})
Dataset({
    features: ['train-file-id', 'speaker-id', 'label'],
    num_rows: 9172
})


In [ ]:
from transformers import AutoConfig, Wav2Vec2Processor

model_name_or_path = "aliyzd95/wavlm-deepmine-base-plus"
save_dir = "aliyzd95/wavlm-deepmine-base-plus-sv"

# config
config = AutoConfig.from_pretrained(
    'microsoft/wavlm-base-plus-sv',
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)
setattr(config, 'pooling_mode', "mean")

processor = Wav2Vec2Processor.from_pretrained('aliyzd95/wavlm-deepmine-base-plus')
target_sampling_rate = processor.feature_extractor.sampling_rate
print(f"The target sampling rate: {target_sampling_rate}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/58.6k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

The target sampling rate: 16000


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

train_df, test_df = train_test_split(df, test_size=0.05, random_state=42)

train_df.drop(columns=['phrase-id', 'speaker-id']).to_csv('train_data.csv', index=False)
test_df.drop(columns=['phrase-id', 'speaker-id']).to_csv('test_data.csv', index=False)

In [ ]:
common_voice_train = load_dataset(
    'csv',
    data_files='train_data.csv',
    split='train',
    streaming=True,
    delimiter=',',
)

common_voice_test = load_dataset(
    'csv',
    data_files='test_data.csv',
    split='train',
    delimiter=',',
)

In [ ]:
import torchaudio
import librosa
import numpy as np

target_sampling_rate = 16_000

def speech_file_to_array(batch):

    speech_array, sampling_rate = torchaudio.load(os.path.join(wav_dir, 'train', f"{batch['train-file-id']}.wav"))
    speech_array = speech_array.squeeze().numpy()
    speech_array = librosa.resample(y=np.asarray(speech_array),
                                    orig_sr=sampling_rate,
                                    target_sr=target_sampling_rate)

    batch["speech"] = speech_array
    batch["sampling_rate"] = target_sampling_rate
    batch["duration_in_seconds"] = len(batch["speech"]) / target_sampling_rate


    return batch

In [ ]:
common_voice_train = common_voice_train.map(speech_file_to_array, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(speech_file_to_array, remove_columns=common_voice_test.column_names)

In [ ]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    result = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0], padding=True)
    result["labels"] = batch['speaker-id-encoded']

    return result

In [ ]:
common_voice_train = common_voice_train.map(prepare_dataset,
                                            remove_columns=common_voice_train.column_names,
                                            batch_size=8,
                                            batched=True,
                                            # num_proc=4
                                            )

common_voice_test = common_voice_test.map(prepare_dataset,
                                          remove_columns=common_voice_test.column_names,
                                          batch_size=8,
                                          batched=True,
                                          # num_proc=4
                                          )

In [ ]:
idx = 0
print(f"Training input_values: {len(common_voice_train[idx]['input_values'])}")
print(f"Training attention_mask: {len(common_voice_train[idx]['attention_mask'])}")
print(f"Training labels: {common_voice_train[idx]['labels']}")

In [ ]:
import torch
from dataclasses import dataclass
from typing import Dict, List, Union
from transformers import Wav2Vec2Processor

@dataclass
class DataCollatorForSpeakerVerification:
    """
    Data collator for speaker verification tasks that will dynamically pad the inputs received.

    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`):
            The processor used for processing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, optional, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences.
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # Extract input values and speaker labels from the batch
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        labels = [feature["labels"] for feature in features]

        # Ensure inputs are padded
        batch = self.processor.pad(
            input_features,
            padding=self.padding,  # Ensure padding is activated
            return_tensors="pt"
        )

        # Convert labels to tensor
        batch["labels"] = torch.tensor(labels)

        return batch


data_collator = DataCollatorForSpeakerVerification(processor=processor, padding=True)

In [ ]:
import numpy as np
from sklearn.metrics import roc_curve
from sklearn.metrics.pairwise import cosine_similarity

def compute_eer(labels, scores):
    # Unpack the embeddings from the scores tuple
    embeddings1, embeddings2 = scores  # Both are of shape (13, 512)

    # Calculate pairwise cosine similarity between embeddings1 and embeddings2
    similarities = cosine_similarity(embeddings1, embeddings2)

    # Create binary labels for same/different speaker (1 for same, 0 for different)
    # Since labels are speaker IDs, we compare each pair of speakers
    binary_labels = np.array([1 if labels[i] == labels[j] else 0 for i in range(len(labels)) for j in range(len(labels))])

    # Flatten similarities to match the binary_labels array
    similarity_scores = similarities.flatten()

    # Compute False Acceptance Rate (FAR) and False Rejection Rate (FRR)
    fpr, tpr, thresholds = roc_curve(binary_labels, similarity_scores, pos_label=1)

    # False Rejection Rate (FRR) is 1 - True Positive Rate (TPR)
    fnr = 1 - tpr

    # Find the threshold where FPR == FNR (Equal Error Rate)
    eer_threshold = thresholds[np.nanargmin(np.abs(fnr - fpr))]
    eer = fpr[np.nanargmin(np.abs(fnr - fpr))]

    return eer, eer_threshold

# Wrapping it in a callable function for Hugging Face Trainer
def compute_metrics(pred):
    labels = pred.label_ids  # Speaker IDs (vector of size 13)
    scores = pred.predictions  # Tuple of two (13, 512) arrays representing embeddings

    # Compute EER
    eer, eer_threshold = compute_eer(labels, scores)

    # Return EER as a metric
    return {
        'eer': eer,
        'eer_threshold': eer_threshold
    }

In [ ]:
import os
import torch

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
torch.cuda.empty_cache()

In [ ]:
from transformers import WavLMForXVector

model = WavLMForXVector.from_pretrained(
    model_name_or_path,
    config=config,
)

Some weights of the model checkpoint at aliyzd95/wavlm-deepmine-base-plus were not used when initializing WavLMForXVector: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing WavLMForXVector from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing WavLMForXVector from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of WavLMForXVector were not initialized from the model checkpoint at aliyzd95/wavlm-deepmine-base-plus and are newly initialized: ['classifier.bias', 'classifier.weight', 'feature_extractor.bias', 'feature_extractor.weight', 'layer_weights', 'objective.weight', 'projector.bias', 'projector.weight', 'tdnn.0.kernel.bias', 'tdnn.0.kernel.weight', 'tdnn.1.kernel.bia

In [ ]:
model.objective.margin = 0.2
model.objective.scale = 32.0

In [ ]:
model.freeze_feature_extractor()
model.gradient_checkpointing_enable()

/usr/local/lib/python3.10/dist-packages/transformers/models/wavlm/modeling_wavlm.py:1733: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=save_dir,
    group_by_length=False,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    # evaluation_strategy="no",
    max_steps = 50000,
    num_train_epochs=3,
    gradient_checkpointing=True,
    fp16=True,
    weight_decay=0.01,
    save_steps=400,
    eval_steps=400,
    logging_steps=400,
    learning_rate=1e-5,
    warmup_steps=500,
    save_total_limit=2,
    push_to_hub=True,
    load_best_model_at_end=True,
    metric_for_best_model="eer",
    greater_is_better=False
    max_grad_norm=1.0,  # This sets gradient clipping
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1541: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=1e-5)

from transformers import get_scheduler

lr_scheduler = get_scheduler(
    name="reduce_lr_on_plateau",
    optimizer=optimizer,
    num_warmup_steps=500,
    num_training_steps=50000,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor,
    optimizers=(optimizer, lr_scheduler),
)

trainer.train()

In [ ]:
if last_checkpoint:
    print(f"last_checkpoint: {last_checkpoint}")
    train_result = trainer.train(resume_from_checkpoint=last_checkpoint)
else:
    train_result = trainer.train()

In [ ]:
processor.push_to_hub(save_dir)

CommitInfo(commit_url='https://huggingface.co/aliyzd95/wavlm-deepmine-large-sv/commit/5110c94ec45685829b1356e3ca55619a815eb693', commit_message='Upload processor', commit_description='', oid='5110c94ec45685829b1356e3ca55619a815eb693', pr_url=None, pr_revision=None, pr_num=None)